# Law Textbook DB

In [ ]:
# set parameters

file = open("info/api.txt", "r")
api_key = file.read()
file.close()
file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()
file = open("info/resultspath.txt", "r")
results_path = file.read()
file.close()

In [ ]:
from langchain_upstage import UpstageLayoutAnalysisLoader
import os
from langchain_upstage import UpstageEmbeddings

UPSTAGE_API_KEY = api_key

# Only split "civil_procedure_casebook" into chunks of 4000 tokens
booklist = ["jurisprudence", "international_law", "criminal_law_casebook", "torts_casebook", "civil_procedure_casebook"]
book = booklist[4]

layzer = UpstageLayoutAnalysisLoader(api_key=UPSTAGE_API_KEY,file_path=os.path.join(data_path, f'law_textbook/{book}.pdf'), output_type="text")
docs = layzer.load()  # or layzer.lazy_load()


# Query-specific embedding model
query_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-query")
# Sentence-specific embedding model
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")

## 2. Text Split

In [ ]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

chunk_size = 5000
chunk_overlap = chunk_size * 0.1

# 2. Split
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap, language=Language.HTML
)
splits = text_splitter.split_documents(docs)
print("Splits:", len(splits))

In [ ]:
print(splits[5])
len(splits[1].page_content)

In [ ]:
chunk = []
for index in range(5,len(splits)) : # Get the context
    chunk.append(splits[index].page_content)

    # Exclude the initial splits as they are irrelevant to the content.

In [ ]:
embedded_documents = passage_embeddings.embed_documents(chunk)

In [ ]:
embedded_documents

In [ ]:
import numpy as np

np.save(data_path+f'embedding/chunked{chunk_size}_Textbook_{book}', np.array(chunk))
np.save(data_path+f'embedding/embedded{chunk_size}_Textbook_{book}', np.array(embedded_documents))

print("1D list saved successfully!")

How to load the saved embedding list:
```
# Load the .npy file (type == numpy)
loaded_array = np.load('my_list.npy')

# Convert the NumPy array to a Python list
restored_list = loaded_array.tolist()

```

In [ ]:
import numpy as np

booklist = ["jurisprudence", "international_law", "criminal_law_casebook", "torts_casebook", "civil_procedure_casebook"]

full_law = []
full_law_embed = []
for book in booklist[:-1] :
    # Load the .npy file (type == numpy)
    textbookDB = np.load(data_path+f'embedding/chunked5000_Textbook_{book}.npy')
    textbookDB = textbookDB.tolist()
    print(len(textbookDB))

    textbookDB_embed = np.load(data_path+f'embedding/embedded5000_Textbook_{book}.npy')
    textbookDB_embed = textbookDB_embed.tolist()

    for idx in range(len(textbookDB)) :
        full_law.append(textbookDB[idx])
        full_law_embed.append(textbookDB_embed[idx])

textbookDB = np.load(data_path+f'embedding/chunked4000_Textbook_civil_procedure_casebook.npy')
textbookDB = textbookDB.tolist()
print(len(textbookDB))

textbookDB_embed = np.load(data_path+f'embedding/embedded4000_Textbook_civil_procedure_casebook.npy')
textbookDB_embed = textbookDB_embed.tolist()

for idx in range(len(textbookDB)) :
    full_law.append(textbookDB[idx])
    full_law_embed.append(textbookDB_embed[idx])


np.save(data_path+f'embedding/full_law_textbook', np.array(chunk))
np.save(data_path+f'embedding/full_law_textbook_embed', np.array(embedded_documents))

In [ ]:
len(full_law)